In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [2]:
DATASET_DIR = "C:/Users/KIIT/Desktop/PetImages"
IMG_SIZE = 64  

In [3]:
CATEGORIES = ["Cat", "Dog"] 

In [4]:
def load_data():
    data = []
    labels = []
    
    for category in ["Cat", "Dog"]:
        path = os.path.join(DATASET_DIR, category)
        class_label = 0 if category == "Cat" else 1
        
        for img_name in tqdm(os.listdir(path)):
            img_path = os.path.join(path, img_name)

            # Ensure the file exists
            if not os.path.exists(img_path):
                print(f"File not found: {img_path}")
                continue
            
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load image in grayscale

            # Skip corrupt or unreadable images
            if img is None:
                print(f"Skipping corrupt image: {img_path}")
                continue
            
            img = cv2.resize(img, (64, 64))  # Resize image
            data.append(img)
            labels.append(class_label)
    
    print(f"Loaded {len(data)} images successfully!")
    return np.array(data), np.array(labels)

# Run and test the function
X, y = load_data()
print(f"X shape: {X.shape}")  # Should print (num_samples, 64, 64)


100%|██████████| 12444/12444 [00:13<00:00, 891.49it/s]


Loaded 24913 images successfully!
X shape: (24913, 64, 64)


In [5]:
import os
import cv2

DATASET_DIR = "C:/Users/KIIT/Desktop/PetImages"

for category in ["Cat", "Dog"]:
    folder = os.path.join(DATASET_DIR, category)

    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)  # Define img_path properly

        # Try to read the image
        img = cv2.imread(img_path)

        # If the image is unreadable, delete it
        if img is None:
            print(f"Removing corrupted image: {img_path}")
            os.remove(img_path)  # Delete corrupt file


In [6]:
X = X / 255.0  # Scale pixel values to [0,1]
X = X.reshape(X.shape[0], -1)  # Flatten images for SVM input

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
print("Dataset Loaded Successfully!")
print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")

Dataset Loaded Successfully!
Training samples: 19930, Testing samples: 4983


In [9]:
X_train_small = X_train[:500]  # Use only 500 samples
y_train_small = y_train[:500]

svm_model = SVC(kernel='linear')
svm_model.fit(X_train_small, y_train_small)


SVC(kernel='linear')

In [10]:
y_pred = svm_model.predict(X_test)

In [11]:
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Model Accuracy: {accuracy:.2f}")


SVM Model Accuracy: 0.51


In [12]:
def predict_image(img_path, model):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0  # Normalize
    img = img.flatten().reshape(1, -1)  # Flatten and reshape
    
    prediction = model.predict(img)
    return "Cat" if prediction == 0 else "Dog"

In [40]:
import cv2
import numpy as np

IMG_SIZE = 64  # Define the expected image size

def predict_image(img_path, model):
    # Load the image in grayscale
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Check if the image was loaded correctly
    if img is None:
        print(f"Error: Could not load image at {img_path}. Please check the file path!")
        return None

    # Resize and preprocess the image
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0  # Normalize
    img = img.flatten().reshape(1, -1)  # Flatten and reshape

    # Predict the label
    prediction = model.predict(img)
    label = "Cat" if prediction[0] == 0 else "Dog"
    
    return label

# ✅ Use a correct file path
test_img = "C:/Users/KIIT/Desktop/PetImages/Cat/0.jpg"
result = predict_image(test_img, svm_model)
if result:
    print(f"Prediction: {result}")


Prediction: Cat
